# Dados - Q3

In [1]:
import pandas as pd
import numpy as np
import xarray as xr

# Buscando os dados no CPTEC

In [2]:
import urllib
url = 'http://ftp.cptec.inpe.br/modelos/tempo/MERGE/GPM/DAILY/2021/09/MERGE_CPTEC_20210928.grib2'
temp_grib = "MERGE_CPTEC_20210928.grib2"
urllib.request.urlretrieve(url, temp_grib)

('MERGE_CPTEC_20210928.grib2', <http.client.HTTPMessage at 0x7f1198e2b070>)

## Lendo o arquivo GRIB2 e transformando pra netCDF

In [3]:
# OBS.: foi necessário instalar a engine 'pynio'. 
# < conda install -c conda-forge pynio >
ds = xr.load_dataset(temp_grib, engine='pynio')
ds.to_netcdf('MERGE_CPTEC_20210928.nc')

In [6]:
# Uma vez feita a conversão, a leitura pode ser feita direto com
ds = xr.open_dataset('MERGE_CPTEC_20210928.nc') # redundante, porém julguei como uma facilidade para rodar a API sempre que reiniciar o kernel
ds

<xarray.Dataset>
Dimensions:           (lat_0: 924, lon_0: 1001)
Coordinates:
  * lat_0             (lat_0) float32 -60.05 -59.95 -59.85 ... 32.05 32.15 32.25
  * lon_0             (lon_0) float32 239.9 240.1 240.1 ... 339.8 339.9 340.0
Data variables:
    PRMSL_P0_L1_GLL0  (lat_0, lon_0) float32 ...
    PREC_P0_L1_GLL0   (lat_0, lon_0) float32 ...

## Recortes
- Recorte a área total da bacia do rio Tietê e salve o resultado em um novo NC (1 pt).
  - nota: utilize o shapefile disponibilizado.

In [9]:
def longitude_formatter(val=float):
    '''
    Adapta longitude de 0 > 360 para -180 > 180
    
    Valores entre 0 e 180 são graus a Leste, estes se mantém.
    Enquanto se forem entre 0 e 180, subtrai-se 360.

    
    Call signature::

            longitude_formatter(val)

    **Args:
    ----------
    val: float


    Returns:
    -------
    obj: <float>
    '''
    
    if val > 180:
        val -= 360
    return val

In [10]:
import geopandas as gpd

In [11]:
data = ds.PREC_P0_L1_GLL0.to_dataframe().reset_index()
data['lon_0'] = data['lon_0'].apply(lambda x: longitude_formatter(x))

In [12]:
gdf = gpd.GeoDataFrame(data,geometry=gpd.points_from_xy(data['lon_0'],data['lat_0']))
gdf

,lat_0,lon_0,PREC_P0_L1_GLL0,geometry
0,-60.049999,-120.050003,0.00,POINT (-120.05000 -60.05000)
1,-60.049999,-119.949997,4.50,POINT (-119.95000 -60.05000)
2,-60.049999,-119.850006,5.25,POINT (-119.85001 -60.05000)
3,-60.049999,-119.750000,6.00,POINT (-119.75000 -60.05000)
4,-60.049999,-119.649994,6.00,POINT (-119.64999 -60.05000)
...,...,...,...,...
924919,32.250000,-20.449982,0.00,POINT (-20.44998 32.25000)
924920,32.250000,-20.349976,0.00,POINT (-20.34998 32.25000)
924921,32.250000,-20.250000,0.00,POINT (-20.25000 32.25000)
924922,32.250000,-20.149994,0.00,POINT (-20.14999 32.25000)


In [13]:
mascara = gpd.read_file('tiete.shp')

In [14]:
mascara.crs

<Geographic 2D CRS: EPSG:4674>
Name: SIRGAS 2000
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: Latin America - SIRGAS 2000 by country
- bounds: (-122.19, -59.87, -25.28, 32.72)
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [16]:
tiete_clip = gpd.clip(gdf = gdf, mask = mascara['geometry'])

tiete_precip = tiete_clip.set_index(['lat_0','lon_0']).drop('geometry', axis=1)

<ipython-input-16-453af9399136>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4674

  tiete_clip = gpd.clip(gdf = gdf, mask = mascara['geometry'])


In [17]:
tiete_precip

PREC_P0_L1_GLL0
lat_0  lon_0                      
-23.85 -46.850006              0.0
       -46.750000              0.0
       -46.649994              0.0
       -46.549988              0.0
-23.75 -47.750000              0.0
...                            ...
-20.85 -49.750000              0.0
       -49.549988              0.0
-20.75 -50.049988              0.0
       -49.949982              0.0
       -49.850006              0.0

[544 rows x 1 columns]

In [18]:
ds_tiete = xr.Dataset.from_dataframe(tiete_precip)
ds_tiete.to_netcdf("TIETE_MERGE_CPTEC_20210928.nc")
ds_tiete

<xarray.Dataset>
Dimensions:          (lat_0: 32, lon_0: 46)
Coordinates:
  * lat_0            (lat_0) float64 -23.85 -23.75 -23.65 ... -20.85 -20.75
  * lon_0            (lon_0) float64 -50.25 -50.15 -50.05 ... -45.85 -45.75
Data variables:
    PREC_P0_L1_GLL0  (lat_0, lon_0) float32 nan nan nan nan ... nan nan nan nan

- Reduza a resolução dos dados para metade da resolução original e salve o resultado em um novo NC (1.5 pt).
  - nota: crie uma grade mais grosseira.

In [20]:
def grid_interp(array):
    '''
    Cria um novo array com metade dos valores do array de entrada.
    O valor da primeira posição é armazenado no novo array enquanto o valor da segunda posição é deixado de lado.
    Esse padrão segue até o final do array de entrada.

    Call signature::

            grid_interp(array)

    **Args:
    ----------
    array: np.array or list


    Returns:
    -------
    obj: list
    '''
    new_array = list()

    count = 0 
    while count < len(array):
        new_array.append(array[count])
        count += 2
    return new_array

In [31]:
# Criando os novos arrays de latitude e longitude para metade da quantidade de valores do dado original 
lats0 = ds['lat_0'].values
lons0 = ds['lon_0'].values
new_lat = grid_interp(lats0)
new_lon = grid_interp(lons0)

In [32]:
# Agora aplicando a interpolação com os novos arrays de lat e lon
ds_tiete_interp = ds_tiete.interp(lat_0=new_lat, lon_0=new_lon)
ds_tiete_interp.to_netcdf('G2_TIETE_MERGE_CPTEC_20210928.nc')
ds_tiete_interp

<xarray.Dataset>
Dimensions:          (lat_0: 462, lon_0: 501)
Coordinates:
  * lat_0            (lat_0) float32 -60.05 -59.85 -59.65 ... 31.75 31.95 32.15
  * lon_0            (lon_0) float32 239.9 240.1 240.4 ... 339.6 339.8 340.0
Data variables:
    PREC_P0_L1_GLL0  (lat_0, lon_0) float64 nan nan nan nan ... nan nan nan nan

- Crie uma listagem de todos os pontos de grade (do arquivo original) dentro da bacia do rio Tietê e salve a listagem em uma tabela (1 pt).

In [43]:
gridPoint_list = tiete_clip.drop(['lat_0', 'lon_0', 'PREC_P0_L1_GLL0'], axis=1)
gridPoint_list.to_csv('gridPoint_list.csv')

In [44]:
gridPoint_list

,geometry
363094,POINT (-46.85001 -23.85000)
363095,POINT (-46.75000 -23.85000)
363096,POINT (-46.64999 -23.85000)
363097,POINT (-46.54999 -23.85000)
364086,POINT (-47.75000 -23.75000)
...,...
393095,POINT (-49.75000 -20.85000)
393097,POINT (-49.54999 -20.85000)
394093,POINT (-50.04999 -20.75000)
394094,POINT (-49.94998 -20.75000)
